In [2]:
from torch.utils.data import Dataset
import torch
from torchvision import transforms
from PIL import Image
import skimage.io
import numpy as np
import pandas as pd
import pretrainedmodels
from torch.utils.data import DataLoader
import time
import glob
import os
from multiprocessing import Pool

# Dataloader and Dataset Object

In [5]:
model_configs = {'se_resnext50_32x4d':{
                   'input_size': 224,
                   'input_mean': [0.485, 0.456, 0.406],
                   'input_std' : [0.229, 0.224, 0.225]
                    },
                 'resnet34':{
                   'input_size': 224,
                   'input_mean': [0.485, 0.456, 0.406],
                   'input_std' : [0.229, 0.224, 0.225]
                    },
                 'dpn68':{
                   'input_size': 224,
                   'input_mean': [0.485, 0.456, 0.406],
                   'input_std' : [0.229, 0.224, 0.225]
                    },
                 'nasnetamobile':{
                   'input_size': 224,
                   'input_mean': [0.5],
                   'input_std' : [0.5]
                    },
                 'bninception':{
                   'input_size': 299,
                   'input_mean': [0.5],
                   'input_std' : [0.5]
                    },
                 'xception':{
                   'input_size': 299,
                   'input_mean': [0.5],
                   'input_std' : [0.5]
                    }
                }




class AtlasData(Dataset):
    def __init__(self, split, train = True, model = 'bninception'):
        self.split = split
        self.train = train
        self.train_str = 'train' if self.train else 'test'
        self.text_file = 'data/atlas_{}_split_{}.txt'.format(self.train_str, self.split)
        
        self.data = [[y for y in x.strip().split(' ')] for x in open(self.text_file, 'r').readlines()]
        self.imgs = [x[0] for x in self.data]
        self.labels = [[int(p) for p in x[1:]] for x in self.data]
        
        self.input_size = model_configs[model]['input_size']
        self.input_mean = model_configs[model]['input_mean']
        self.input_std = model_configs[model]['input_std']
        
        self.transforms = transforms.Compose([transforms.RandomRotation(180),
                                              transforms.Resize(self.input_size),
                                              transforms.ToTensor(),
                                              transforms.Normalize(self.input_mean, self.input_std),
                                            ])
        
    def dump_image(self, i):
        image = self.load_image_stack(self.imgs[i])
        image_name = "data/train/{}_{}.png".format(self.imgs[i], 'stacked')
        image.save(image_name)
        print("Saved", image_name)

    
    def load_image_stack(self, image_id):
        colors = ['red', 'green', 'blue', 'yellow']
        absolute_paths = ["data/train/{}_{}.png".format(image_id, color) for color in colors]
        
        images = [skimage.io.imread(path) for path in absolute_paths]
        
        image_red = images[0]
        image_green = images[1] + (images[3]/2).astype(np.uint8)
        image_blue = images[2] + (images[3]/2).astype(np.uint8)
        
        final_image = np.stack((image_red, image_green, image_blue), -1)
        to_display = Image.fromarray(final_image)
        return to_display
        
    def load_image(self, i):
        image_id = self.imgs[i]
        image_path = "data/train/{}_{}.png".format(image_id, 'stacked')
        image = Image.open(image_path)
        
        return image

    
    def __getitem__(self, i):
        image = self.load_image(i)
        image = self.transforms(image)
        
        label = self.labels[i]
        label_arr = np.zeros(28, dtype = np.float32)
        [np.put(label_arr, x, 1) for x in label]
        
        label_arr = torch.from_numpy(label_arr)
        
        return image, label_arr, label
        
    def __len__(self):
        return len(self.imgs)

# Training Script

In [7]:
import pretrainedmodels
from torch.utils.data import DataLoader
import time
from sklearn.metrics import f1_score

def f1_micro(y_true, y_preds, thresh=0.5, eps=1e-20):
    preds_bin = y_preds > thresh # binary representation from probabilities (not relevant)
    truepos = preds_bin * y_true
    
    p = truepos.sum() / (preds_bin.sum() + eps) # take sums and calculate precision on scalars
    r = truepos.sum() / (y_true.sum() + eps) # take sums and calculate recall on scalars
    
    f1 = 2*p*r / (p+r+eps) # we calculate f1 on scalars
    return f1

def f1_macro(y_true, y_preds, thresh=0.5, eps=1e-20):
    preds_bin = y_preds > thresh # binary representation from probabilities (not relevant)
    truepos = preds_bin * y_true

    p = truepos.sum(axis=0) / (preds_bin.sum(axis=0) + eps) # sum along axis=0 (classes)
                                                            # and calculate precision array
    r = truepos.sum(axis=0) / (y_true.sum(axis=0) + eps)    # sum along axis=0 (classes) 
                                                            #  and calculate recall array

    f1 = 2*p*r / (p+r+eps) # we calculate f1 on arrays
    return np.mean(f1)


class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = torch.nn.functional.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.sum(dim=1).mean()

def train_and_val(model_name, split, batch_size, epochs, lr, start_epoch):
    
    model = pretrainedmodels.__dict__[model_name](num_classes = 1000)
    num_features = model.last_linear.in_features 
    model.last_linear = torch.nn.Linear(num_features, 28)
    
    if model_name == 'polynet':
        model = torch.nn.DataParallel(model, device_ids = [0,1,2]).cuda()
 
    
    if glob.glob('{}_0*'.format(model_name)):
        if model_name == 'vgg19_bn':
            pth_file = torch.load('vgg19_bn_0_40.pth.tar')
        else:
            pth_file = torch.load('{}_0.pth.tar'.format(model_name))
        state_dict = pth_file['state_dict']
        model.load_state_dict(state_dict)
        start_epoch = pth_file['epoch']
        
    if model_name != 'polynet':
        model.cuda()

    train_dataset = AtlasData(split = split, train = True, model = model_name)
    val_dataset = AtlasData(split = split, train = False, model = model_name)

    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
    val_loader = DataLoader(val_dataset, batch_size = 1, shuffle = False)
    
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)   
    
    
    for epoch in range(start_epoch,epochs+1):
        
        train(model, train_loader, optimizer, criterion, epoch)
        avg_loss, f1_score = validate(model, val_loader, criterion, epoch)
                    
        if epoch % 10 == 0:
            filename = '{}_{}_{}.pth.tar'.format(model_name, split, epoch)
        else:
            filename = '{}_{}.pth.tar'.format(model_name, split)
            
        state = {'loss': avg_loss, 'f1_score': f1_score, 'epoch': epoch+1, 'state_dict': model.state_dict()}           
        torch.save(state, filename)

    
def train(model, train_loader, optimizer, criterion, epoch):
    
    model.train()
    start = time.time()
    losses = []
    for i, (images, label_arrs, labels) in enumerate(train_loader):
        images = images.cuda()
        label_arrs = label_arrs.cuda()

        outputs = model(images)

        loss = criterion(outputs, label_arrs)
        losses.append(loss.data[0])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        end = time.time()
        elapsed = end-start
        
        if i%100==0:
            print("Epoch [{}], Iteration [{}/{}], Loss: {:.4f} ({:.4f}), Elapsed Time {:.4f}"
                .format(epoch, i+1, len(train_loader), loss.data[0], sum(losses)/len(losses), elapsed))
            
    print("Average Loss: {}".format(sum(losses)/len(losses)))
            

def validate(model, val_loader, criterion, epoch):
    
    model.eval()
    
    losses = []
    y_pred = np.zeros(len(val_loader) * 28).reshape(len(val_loader), 28)
    y_true = np.zeros(len(val_loader) * 28).reshape(len(val_loader), 28)

    for i, (images, label_arrs, labels) in enumerate(val_loader):
        images = images.cuda()
        label_arrs_cuda = label_arrs.cuda()

        raw_predictions = model(images)
        outputs = raw_predictions.data
        
        loss = criterion(outputs, label_arrs_cuda)
        losses.append(loss.data)
        
        predictions = np.arange(28)[raw_predictions.data[0] > 0.15]
        
        y_pred[i,:] = predictions
        y_true[i,:] = label_arrs
        
        if sum(predictions) == 0:
            prediction = np.argmax(raw_predictions.detach().cpu().numpy())
            predictions = np.zeros(28)
            np.put(predictions, prediction, 1)
        
        
        if i%1000==0:
            print('Testing {}/{}: Loss {}'.format(i, 
                                                 len(val_loader), 
                                                 sum(losses)/len(losses)))
                                                                         
    score = f1_macro(y_true, y_pred)
    avg_loss = sum(losses)/len(losses)
    print("Avg Loss {}".format(avg_loss))
    print("Score {}".format(score))

    return avg_loss, score

split = 0
batch_size = 16
epochs = 100
lr = 0.0001
model_list = [#'xception',
#               'se_resnext50_32x4d', 
#               'nasnetamobile',
              'bninception', 
              'resnet34']

start_epoch = 1

for model_name in model_list:
    print("Training {}".format(model_name))
    train_and_val(model_name, split, batch_size, epochs, lr, start_epoch)

Training bninception
inception_3a_double_3x3_reduce_bn.bias
inception_5a_pool_proj_bn.weight
inception_4d_double_3x3_2_bn.bias
inception_4a_pool_proj_bn.weight
inception_3a_double_3x3_2_bn.running_mean
inception_5a_double_3x3_reduce_bn.weight
inception_3a_double_3x3_reduce_bn.weight
inception_3b_3x3_reduce_bn.running_mean
inception_4e_3x3_reduce_bn.running_var
inception_4d_pool_proj_bn.running_mean
inception_3a_3x3_reduce_bn.weight
inception_4c_1x1_bn.bias
conv1_7x7_s2_bn.weight
inception_4a_1x1_bn.running_var
conv2_3x3_bn.weight
inception_3a_pool_proj_bn.weight
inception_3b_1x1_bn.running_mean
inception_4b_3x3_bn.bias
inception_4c_pool_proj_bn.bias
inception_3b_double_3x3_2_bn.bias
inception_4d_pool_proj_bn.running_var
inception_4c_3x3_bn.running_mean
inception_5b_3x3_bn.running_mean
inception_3a_3x3_reduce_bn.running_var
inception_5b_pool_proj_bn.running_var
inception_4e_3x3_reduce_bn.running_mean
inception_3b_double_3x3_2_bn.weight
inception_4c_3x3_bn.bias
inception_3a_double_3x3_1_

RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58

In [21]:
model = pretrainedmodels.__dict__['bninception']()
model

inception_5a_1x1_bn.running_var
inception_3c_3x3_bn.running_var
inception_4b_pool_proj_bn.bias
inception_3b_double_3x3_2_bn.running_var
inception_3a_double_3x3_2_bn.weight
inception_3b_1x1_bn.bias
inception_4b_double_3x3_1_bn.running_mean
inception_5b_1x1_bn.bias
inception_4d_3x3_reduce_bn.running_mean
inception_4a_3x3_reduce_bn.bias
inception_3b_double_3x3_2_bn.weight
inception_4c_double_3x3_reduce_bn.weight
conv2_3x3_reduce_bn.running_var
conv1_7x7_s2_bn.weight
inception_3b_1x1_bn.running_mean
inception_5a_double_3x3_1_bn.weight
inception_5b_3x3_reduce_bn.running_var
inception_4e_3x3_reduce_bn.running_var
inception_4c_double_3x3_1_bn.bias
inception_3a_double_3x3_reduce_bn.weight
conv1_7x7_s2_bn.running_mean
inception_5a_pool_proj_bn.bias
inception_4a_double_3x3_reduce_bn.running_mean
inception_3a_3x3_reduce_bn.weight
inception_4c_double_3x3_1_bn.running_mean
inception_3b_pool_proj_bn.weight
inception_4c_3x3_bn.running_var
inception_3a_3x3_bn.running_mean
inception_4a_double_3x3_reduc

BNInception(
  (conv1_7x7_s2): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (conv1_7x7_s2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (conv1_relu_7x7): ReLU(inplace)
  (pool1_3x3_s2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=True)
  (conv2_3x3_reduce): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  (conv2_3x3_reduce_bn): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (conv2_relu_3x3_reduce): ReLU(inplace)
  (conv2_3x3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_3x3_bn): BatchNorm2d(192, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (conv2_relu_3x3): ReLU(inplace)
  (pool2_3x3_s2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=True)
  (inception_3a_1x1): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
  (inception_3a_1x1_bn): BatchNorm2d(64, eps=1e-0

# Generate Predictions

In [8]:
class EvalAtlasData(Dataset):
    def __init__(self, model = 'bninception'):
        self.image_ids = sorted(set([x.split('_')[0] for x in os.listdir('data/test')]))
        
        self.input_size = model_configs[model]['input_size']
        self.input_mean = model_configs[model]['input_mean']
        self.input_std = model_configs[model]['input_std']
        
        self.transforms = transforms.Compose([transforms.Resize(self.input_size),
                                              transforms.ToTensor(),
                                              transforms.Normalize(self.input_mean, self.input_std),
                                            ])
        
        
    def load_image_stack(self, image_id):
        colors = ['red', 'green', 'blue', 'yellow']
        absolute_paths = ["data/test/{}_{}.png".format(image_id, color) for color in colors]
        
        images = [skimage.io.imread(path) for path in absolute_paths]
        
        image_red = images[0]
        image_green = images[1] + (images[3]/2).astype(np.uint8)
        image_blue = images[2] + (images[3]/2).astype(np.uint8)
        
        final_image = np.stack((image_red, image_green, image_blue), -1)
        to_display = Image.fromarray(final_image)
        return to_display
    
    def dump_image(self, i):
        image = self.load_image_stack(self.image_ids[i])
        image_name = "data/test/{}_{}.png".format(self.image_ids[i], 'stacked')
        image.save(image_name)
        print("Saved", image_name)
        
    def load_image(self, i):
        image_id = self.image_ids[i]
        image_path = "data/test/{}_{}.png".format(image_id, 'stacked')
        image = Image.open(image_path)
        
        return image, image_id
    
    def __len__(self):
        return len(self.image_ids)
        
    def __getitem__(self, i):
        image, image_id = self.load_image(i)
        image = self.transforms(image)
        
        return image_id, image

In [9]:
import torch
import pretrainedmodels
import pandas as pd
from torch.nn import Softmax

def generate_preds(model_name):
    model = pretrainedmodels.__dict__[model_name](num_classes = 1000, pretrained = 'imagenet')
    in_features = model.last_linear.in_features
    model.last_linear = torch.nn.Linear(in_features, 28)
    
    if model_name == 'polynet':
        model = torch.nn.DataParallel(model, device_ids = [0,1,2,3]).cuda()
        model.load_state_dict(torch.load('{}_0.pth.tar'.format(model_name)))
        model = model.eval()
    else:
        model.load_state_dict(torch.load('{}_0.pth.tar'.format(model_name)))
        model = model.eval()
        model.cuda()

    eval_data = EvalAtlasData(model = model_name)
    dataloader = DataLoader(eval_data, 1, False)
    
    preds = []
    for i, (image_id, images) in enumerate(dataloader):
        images = images.cuda()

        raw_predictions = (model(images))
        predictions = np.argwhere(raw_predictions.data[0] > 0.15)
        try:
            num_predictions = len(predictions.data[0])
        except IndexError:
            num_predictions = 0

        print('-----------------------------------------------------')
        print(image_id[0])
        print('Raw Prediction', raw_predictions)
        if num_predictions == 0:
            print('No value passed the threshold')
            predictions = [np.argmax(raw_predictions.detach().cpu().numpy())]
            num_predictions = 1
            print("Prediction:", predictions)
            print("Number of predictions", num_predictions)
        else:
            predictions = predictions.data[0].tolist()
            print("Prediction:", predictions)
            print("Number of predictions", num_predictions)

        predicted = ' '.join('%d' % prediction for prediction in predictions)
        print(image_id[0])
        print(predicted)
        pred = dict(Id = image_id[0], Predicted = predicted)
        preds.append(pred)
        
    df = pd.DataFrame(preds)
    df.to_csv('{}.csv'.format(model_name), index = False)

In [10]:
generate_preds('polynet')

NameError: name 'model_configs' is not defined

In [11]:
baha = "_features.0.weight", "_features.0.bias", "_features.1.weight", "_features.1.bias", "_features.1.running_mean", "_features.1.running_var", "_features.3.weight", "_features.3.bias", "_features.4.weight", "_features.4.bias", "_features.4.running_mean", "_features.4.running_var", "_features.7.weight", "_features.7.bias", "_features.8.weight", "_features.8.bias", "_features.8.running_mean", "_features.8.running_var", "_features.10.weight", "_features.10.bias", "_features.11.weight", "_features.11.bias", "_features.11.running_mean", "_features.11.running_var", "_features.14.weight", "_features.14.bias", "_features.15.weight", "_features.15.bias", "_features.15.running_mean", "_features.15.running_var", "_features.17.weight", "_features.17.bias", "_features.18.weight", "_features.18.bias", "_features.18.running_mean", "_features.18.running_var", "_features.20.weight", "_features.20.bias", "_features.21.weight", "_features.21.bias", "_features.21.running_mean", "_features.21.running_var", "_features.23.weight", "_features.23.bias", "_features.24.weight", "_features.24.bias", "_features.24.running_mean", "_features.24.running_var", "_features.27.weight", "_features.27.bias", "_features.28.weight", "_features.28.bias", "_features.28.running_mean", "_features.28.running_var", "_features.30.weight", "_features.30.bias", "_features.31.weight", "_features.31.bias", "_features.31.running_mean", "_features.31.running_var", "_features.33.weight", "_features.33.bias", "_features.34.weight", "_features.34.bias", "_features.34.running_mean", "_features.34.running_var", "_features.36.weight", "_features.36.bias", "_features.37.weight", "_features.37.bias", "_features.37.running_mean", "_features.37.running_var", "_features.40.weight", "_features.40.bias", "_features.41.weight", "_features.41.bias", "_features.41.running_mean", "_features.41.running_var", "_features.43.weight", "_features.43.bias", "_features.44.weight", "_features.44.bias", "_features.44.running_mean", "_features.44.running_var", "_features.46.weight", "_features.46.bias", "_features.47.weight", "_features.47.bias", "_features.47.running_mean", "_features.47.running_var", "_features.49.weight", "_features.49.bias", "_features.50.weight", "_features.50.bias", "_features.50.running_mean", "_features.50.running_var", "linear0.weight", "linear0.bias", "linear1.weight", "linear1.bias", "last_linear.weight", "last_linear.bias"

In [12]:
cumbray = list(baha)

In [13]:
state = torch.load('vgg19_bn_0.pth.tar')
state_dict = state['state_dict']

In [14]:
for cum in cumbray:
    state_dict[cum] = None

In [15]:
state['state_dict'] = state_dict

In [16]:
torch.save(state, 'vgg19_bn_0.pth.tar')

# Agenda

1. Use Focal Loss https://www.kaggle.com/iafoss/pretrained-resnet34-with-rgby-fast-ai
2. Somehow use the Y channel

Trained:
1. InceptionV4 100 epochs
2. SE-ResNext 33 epochs
3. PolyNet 33 epochs

In [10]:
import pretrainedmodels
import torch

model = pretrainedmodels.__dict__['vgg19_bn']()
in_features = model.last_linear.in_features

model.last_linear = torch.nn.Linear(in_features, 28)
model.cuda()
model.load_state_dict(torch.load('vgg19_bn_0_40.pth.tar')['state_dict'])

RuntimeError: Error(s) in loading state_dict for VGG:
	Unexpected key(s) in state_dict: "_features.0.weight", "_features.0.bias", "_features.1.weight", "_features.1.bias", "_features.1.running_mean", "_features.1.running_var", "_features.3.weight", "_features.3.bias", "_features.4.weight", "_features.4.bias", "_features.4.running_mean", "_features.4.running_var", "_features.7.weight", "_features.7.bias", "_features.8.weight", "_features.8.bias", "_features.8.running_mean", "_features.8.running_var", "_features.10.weight", "_features.10.bias", "_features.11.weight", "_features.11.bias", "_features.11.running_mean", "_features.11.running_var", "_features.14.weight", "_features.14.bias", "_features.15.weight", "_features.15.bias", "_features.15.running_mean", "_features.15.running_var", "_features.17.weight", "_features.17.bias", "_features.18.weight", "_features.18.bias", "_features.18.running_mean", "_features.18.running_var", "_features.20.weight", "_features.20.bias", "_features.21.we